In [ ]:
# Install the required packages for the project

%pip install -r requirements.txt

In [ ]:
# Ready the environment variables from the .env file

import os
from dotenv import load_dotenv

load_dotenv(override=True) 

TOKENS = os.getenv("TOKENS", "").split(",")

In [ ]:
# Define constants for API usage

API_URL = 'https://api.github.com'

LIMIT_REQUESTS_BY_TOKEN_PER_HOUR = 5000
LIMIT_REQUESTS_BY_IP_PER_HOUR = 60

In [ ]:
# Set API request utils (RUN IT ONLY ONCE TO KEEP COUNTER SYNCED)

import requests

# Initialize the counter for each token
counter = {token: LIMIT_REQUESTS_BY_TOKEN_PER_HOUR for token in TOKENS}
counter.update({None: LIMIT_REQUESTS_BY_IP_PER_HOUR})

# Get the next avaliable token to use for the request
def get_avaliable_token(): 
    token = None
    for t in counter:
        if counter[t] > 0:
            token = t
            break
        else:
            invalidate_token(t)
    
    return token
    
# Invalidate a token if it results in a error, avoiding to use it again
def invalidate_token(token):
    if token in counter:
        del counter[token]
        print(f"Token {token} has been invalidated.")
    else:
        print(f"Token {token} not found in the list.")

# Mark a token as used, reducing the counter
def used_token(token):
    if token in counter:
        counter[token] -= 1
        # print(f"Token {token} used, remaining requests: {counter[token]}")
        if counter[token] == 0:
            invalidate_token(token)
    else:
        print(f"Token {token} not found in the list.")

# Function to make a request to the API
def request (path):
    # Set the URL to the API endpoint
    url = f'{API_URL}/{path}'
    if (path.startswith(API_URL)):
        url = path

    # Define headers with the token
    token = get_avaliable_token()
    headers = {}
    if token is not None:
        headers['Authorization'] = f'Bearer {token}'

    # Make the GET request
    response = requests.get(url, headers=headers)
    if response.ok:
        # print(f"Request successful for token: {token}")
        used_token(token)
    else:
        if response.status_code == 401:
            print(f"Unauthorized access for token: {token}")
            used_token(token)

        elif response.status_code == 403:
            print(f"Rate limit exceeded for token: {token}")
            invalidate_token(token)
            # Retry with the next available token
            if (token is not None):
                print("Retrying with the next available token...")
                return request(path)
            
        elif response.status_code == 404:
            print("Resource not found")

        elif response.status_code == 429:
            print("Rate limit exceeded for IP")
            invalidate_token(token)

    return response

In [ ]:
# Fetch and form/save initial graph

import networkx as nx
import time

# Configuration
SEED_USER = 'gabriel-dp' # Initial node
MAX_REQUESTS = 15000 # Max Requests (generally keep 5k per token)
MAX_DEPTH = 3 # How deep will we search
BLOCKED_USERS = {'torvalds'} # Can desconsider some users
SLEEP_TIME = 0.2  # Delay between requests

# Cache for using less requests
followers_cache = {}
following_cache = {}

# Fetch followers/following (Can limit number of pages)
def get_all_follow_data(username, follow_type='followers'):
    print(f"getting all follow data for {username}")
    if follow_type == 'followers' and username in followers_cache:
        return followers_cache[username]
    if follow_type == 'following' and username in following_cache:
        return following_cache[username]

    per_page = 100
    page = 1
    pages_limit = 10
    results = []

    while True:
        path = f'users/{username}/{follow_type}?per_page={per_page}&page={page}'
        response = request(path)
        if not response or not response.ok:
            break
        data = response.json()
        if not data:
            break
        results.extend([user['login'] for user in data])
        if len(data) < per_page or page >= pages_limit:
            break
        page += 1
        time.sleep(SLEEP_TIME)

    if follow_type == 'followers':
        followers_cache[username] = results
    else:
        following_cache[username] = results

    return results

# Build graph (mutual)
def build_mutual_graph(seed_user, max_requests=10000, max_depth=3):
    G = nx.Graph()
    visited = set()
    queue = [(seed_user, 0)]
    total_requests = 0

    while queue and total_requests < max_requests:
        user, level = queue.pop(0)

        if user in visited or user in BLOCKED_USERS or level > max_depth:
            continue

        print(f"Processing {user} (level {level}) | Total requests: {total_requests}")
        visited.add(user)

        try:
            followers = get_all_follow_data(user, 'followers')
            total_requests += 1
            following = get_all_follow_data(user, 'following')
            total_requests += 1
        except Exception as e:
            print(f"Error fetching data for {user}: {e}")
            continue

        mutuals = set(followers).intersection(following)
        for mutual in mutuals:
            if mutual in BLOCKED_USERS:
                continue
            G.add_edge(user, mutual)

            if mutual not in visited:
                queue.append((mutual, level + 1))

        time.sleep(SLEEP_TIME)

    print(f"Finished: {len(G.nodes())} nodes, {len(G.edges())} edges, {total_requests} requests")
    return G

# Save graph
def save_graph(graph, filename='github_mutual_follow_graph.graphml'):
    nx.write_graphml(graph, filename)
    print(f"Graph saved to {filename}")

# Entry piont
if __name__ == "__main__":
    graph = build_mutual_graph(SEED_USER, MAX_REQUESTS, MAX_DEPTH)
    save_graph(graph)


In [ ]:
# Plot and print the caracterization of the network

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load graph from file
filename = 'github_mutual_follow_graph.graphml'
G_original = nx.read_graphml(filename)
mutual_edges = [(u, v) for u, v in G_original.edges() if G_original.has_edge(v, u)]
G = nx.Graph()
G.add_edges_from(mutual_edges)

print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())

# Plot degree distribution
degree_sequence = [d for n, d in G.degree()]
plt.figure(figsize=(10, 5))
plt.hist(degree_sequence, bins=30, color='skyblue', edgecolor='black')
plt.title("Distribuição de graus")
plt.xlabel("Grau")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

# Calculate clustering
clustering = nx.clustering(G)
avg_clustering = np.mean(list(clustering.values()))
print("Coeficiente de clustering médio:", avg_clustering)

full_clustering_nodes = [n for n, c in clustering.items() if c == 1.0]
print("Número de nós com clustering = 1.0:", len(full_clustering_nodes))

# Plotting degrees of clusterings of 1.0
sizes = [G.degree(n) for n in full_clustering_nodes]
print("Graus médios dos nós com clustering = 1.0:", np.mean(sizes))
plt.hist(sizes, bins=20, color='orange', edgecolor='black')
plt.title("Distribuição de grau dos nós com clustering = 1.0")
plt.xlabel("Grau")
plt.ylabel("Frequência")
plt.show()

# Calculates centralities (degree and eigenvector)
degree_centrality = nx.degree_centrality(G)
try:
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
except nx.PowerIterationFailedConvergence:
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
print("eigenvector centrality done")

# Top 5 by degree centrality
top_5_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 usuários por centralidade de grau:")
for node, centrality in top_5_degree:
    print(f"Nó: {node}, Centralidade de grau: {centrality:.4f}")

# Top 5 by eigenvector centrality
top_5_eigen = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 usuários por centralidade de autovetor:")
for node, centrality in top_5_eigen:
    print(f"Nó: {node}, Centralidade de autovetor: {centrality:.4f}")

# Remove edges with lower than 1 degree for performance and better visualization
low_degree_nodes = [n for n, d in G.degree() if d <= 5]
G.remove_nodes_from(low_degree_nodes)
print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())


# Scaling node size
scaler = MinMaxScaler(feature_range=(5, 150))
deg_vals = np.array([degree_centrality[n] for n in G.nodes()]).reshape(-1, 1)
eig_vals = np.array([eigenvector_centrality[n] for n in G.nodes()]).reshape(-1, 1)
degree_sizes = scaler.fit_transform(deg_vals).flatten()
eigen_sizes = scaler.fit_transform(eig_vals).flatten()
print("scaling done")

# Layout (kamada_kawai or spring)
pos = nx.spring_layout(G, k=1.5, iterations=200, seed=42)
print("positioning done")

# Degree plot
plt.figure(figsize=(14, 10))
nx.draw_networkx_nodes(G, pos, node_size=degree_sizes, alpha=0.8, node_color='teal')
print("drawn nodes")
nx.draw_networkx_edges(G, pos, alpha=0.05, edge_color='gray', width=0.3)
print("drawn edges")
plt.title("Tamanhos proporcionais à centralidade de grau")
plt.axis('off')
plt.show()

# Eigenvector plot
plt.figure(figsize=(14, 10))
nx.draw_networkx_nodes(G, pos, node_size=eigen_sizes, alpha=0.8, node_color='royalblue')
nx.draw_networkx_edges(G, pos, alpha=0.05, edge_color='gray', width=0.3)
plt.title("Tamanhos proporcionais à centralidade de autovetor")
plt.axis('off')
plt.show()

In [ ]:
# Plotting subgraphs

k = 300
# Top k degree graph
top_nodes_degree = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:k]
G_top_degree = G.subgraph(top_nodes_degree)

# Top K eigenvector graph
top_nodes_eigenvector = sorted(eigenvector_centrality, key=eigenvector_centrality.get, reverse=True)[:k]
G_top_eigenvector = G.subgraph(top_nodes_eigenvector)

def scale_sizes(subgraph, centrality_dict):
    scaler = MinMaxScaler(feature_range=(5, 150))
    centrality_vals = np.array([centrality_dict[n] for n in subgraph.nodes()]).reshape(-1, 1)
    scaled_sizes = scaler.fit_transform(centrality_vals).flatten()
    return scaled_sizes

# degree plot
degree_sizes_sub = scale_sizes(G_top_degree, degree_centrality)
# pos_degree = nx.kamada_kawai_layout(G_top_degree)
pos_degree = nx.spring_layout(G_top_degree, k=0.5, iterations=200, seed=42)

plt.figure(figsize=(14, 10))
nx.draw_networkx_nodes(G_top_degree, pos_degree, node_size=degree_sizes_sub, alpha=0.8, node_color='teal')
nx.draw_networkx_edges(G_top_degree, pos_degree, alpha=0.1, edge_color='gray', width=0.4)
plt.title(f"Top {k} Nós - Centralidade de Grau")
plt.axis('off')
plt.show()

# eigenvector plot
eigen_sizes_sub = scale_sizes(G_top_eigenvector, eigenvector_centrality)
# pos_eigen = nx.kamada_kawai_layout(G_top_eigenvector)
pos_eigen = nx.spring_layout(G_top_eigenvector, k=0.5, iterations=200, seed=42)

plt.figure(figsize=(14, 10))
nx.draw_networkx_nodes(G_top_eigenvector, pos_eigen, node_size=eigen_sizes_sub, alpha=0.8, node_color='royalblue')
nx.draw_networkx_edges(G_top_eigenvector, pos_eigen, alpha=0.1, edge_color='gray', width=0.4)
plt.title(f"Top {k} Nós - Centralidade de Autovetor")
plt.axis('off')
plt.show()